# Assignment 1

## Harshit Aggarwal | 2021111015

In [16]:
import numpy as np
import cv2

# Q1.1

In [17]:
image_path = 'calib-object.jpg'
image = cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray_image, maxCorners=300, qualityLevel=0.4, minDistance=10)
if corners is not None:
    corners = (np.intp(corners).reshape(np.intp(corners).shape[0], 2))
    sorted_indices = np.argsort(corners[:, 0])
    corners = corners[sorted_indices]
    # 18 19 28 34 39 41
    required_corners = np.array([1, 2, 18, 19, 28, 34, 39, 41, 89, 82])
    for corner in corners[required_corners]:
        x, y = corner.ravel()
        print(x, y)
        cv2.circle(image, (x, y), 20, (255, 0, 0), -1)
    required_world_coordinates = np.array([[0, -2, 12], [0, 12, 10], [0, 4, 2], [0, 0, 2], [2, 0, 0], [2, 6, 0], [4, 2, 0], [4, 4, 0], [14, 12, 0], [14, -6, 0]])
    required_image_coordinates = corners[required_corners]
    desired_width = 800
    desired_height = 600
    height, width, channels = image.shape
    ratio_width = desired_width / width
    ratio_height = desired_height / height
    ratio = min(ratio_width, ratio_height)
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    resized_img = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    cv2.imshow('Chessboard', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No corners were found")

701 3038
801 1677
1416 2278
1416 2594
1653 2598
1663 2122
1778 2485
1780 2320
2616 1731
2482 3481


# Q1.2

In [19]:
def compute_homography(image_points, world_points):
    A = []
    for i in range(len(image_points)):
        X, Y, Z = world_points[i]
        x, y = image_points[i]
        A.append([X, Y, Z, 1, 0, 0, 0, 0, -x*X, -x*Y, -x*Z, -x])
        A.append([0, 0, 0, 0, X, Y, Z, 1, -y*X, -y*Y, -y*Z, -y])
    A = np.array(A)
    A *= -1
    
    _, s, vh = np.linalg.svd(A)
    m = vh[np.argmin(s), :]
    M = m.reshape(3, 4)
    # print(M)

    MMT = M[:, :3] @ M[:, :3].T
    mm11, mm21, mm13, mm22, mm23 = MMT[0, 0], MMT[1, 0], MMT[0, 2], MMT[1, 1], MMT[1, 2]
    u0 = mm13
    v0 = mm23
    beta = np.sqrt(mm22 - v0**2)
    gamma = (mm21 - u0*v0) / beta
    alpha = np.sqrt(mm11 - gamma**2 - u0**2)
    K = np.array([[alpha, gamma, u0],
                [0,     beta,  v0],
                [0,     0,     1]])
    R = np.linalg.inv(K) @ M[:, :3]
    t = np.linalg.inv(K) @ M[:, 3]
    return K, R, t, M

In [20]:
K, R, t, P = compute_homography(required_image_coordinates, required_world_coordinates)
print("Intrinsic Matrix:\n", K)
print("Rotation Matrix:\n", R)
print("Translation Vector:\n", t)
print("projection Matrix:\n", P)
hh = P @ np.array([0, 4, 2, 1])
print(hh[0]/hh[2], hh[1]/hh[2])

Intrinsic Matrix:
 [[2.87593171e-02 5.51746604e-03 8.45478197e-08]
 [0.00000000e+00 3.08689236e-02 1.10790208e-07]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Rotation Matrix:
 [[-4.04701924e-01 -9.01853873e-02  9.09990631e-01]
 [ 1.83278637e-01  9.66933276e-01  1.77338605e-01]
 [ 5.12849870e-06  1.83432101e-06  4.93656128e-06]]
Translation Vector:
 [-1.26246540e+01 -2.77150907e+01 -3.35281612e-04]
projection Matrix:
 [[-1.06277173e-02  2.74135137e-03  2.71491688e-02 -5.15993498e-01]
 [ 5.65761426e-03  2.98481895e-02  5.47425186e-03 -8.55535018e-01]
 [ 5.12849870e-06  1.83432101e-06  4.93656128e-06 -3.35281612e-04]]
1417.0718594646435 2279.972987237213


# Q1.3

In [21]:
wireframe_points_world = np.array([
    [14, 12, 0],
    [0, 12, 0],
    [14, -6, 0],
    [0, -6, 0]
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(image, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
wireframe_points_world = np.array([
    [0, 12, 14],
    [0, 12, 0],
    [0, -6, 14],
    [0, -6, 0]
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(image, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
desired_width = 800
desired_height = 600
height, width, channels = image.shape
ratio_width = desired_width / width
ratio_height = desired_height / height
ratio = min(ratio_width, ratio_height)
new_width = int(width * ratio)
new_height = int(height * ratio)
resized_img = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
cv2.imshow('Chessboard', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Q1.4

In [22]:
pitch = np.arcsin(-R[2, 0])
roll = np.arctan2(R[2, 1], R[2, 2])
yaw = np.arctan2(R[1, 0], R[0, 0])
pitch_deg = np.degrees(pitch)
roll_deg = np.degrees(roll)
yaw_deg = np.degrees(yaw)
print(f"Pitch (Tilt): {pitch_deg:.2f} degrees")
print(f"Roll: {roll_deg:.2f} degrees")
print(f"Yaw (Pan): {yaw_deg:.2f} degrees")

Pitch (Tilt): -0.00 degrees
Roll: 20.38 degrees
Yaw (Pan): 155.64 degrees


### Explaination:
These values will change according to the world coordinates choosen. These represent the tilt, roll and pan of the camera in the world frame.

# Q2.1

In [23]:
image_path = 'calib-object.jpg'
image = cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray_image, maxCorners=300, qualityLevel=0.4, minDistance=10)
if corners is not None:
    corners = (np.intp(corners).reshape(np.intp(corners).shape[0], 2))
    sorted_indices = np.argsort(corners[:, 0])
    corners = corners[sorted_indices]
    # 18 19 28 34 39 41 38 43
    required_corners = np.array([1, 2, 18, 19, 28, 34, 39, 41, 38, 43, 44, 89, 82])
    for corner in corners[required_corners]:
        x, y = corner.ravel()
        print(x, y)
        cv2.circle(image, (x, y), 20, (255, 0, 0), -1)
    required_world_coordinates = np.array([[0, -2, 12], [0, 12, 10], [0, 4, 2], [0, 0, 2], [2, 0, 0], [2, 6, 0], [4, 2, 0], [4, 4, 0], [4, 0, 0], [4, 8, 0], [4, 10, 0], [14, 12, 0], [14, -6, 0]])
    required_image_coordinates = corners[required_corners]
    desired_width = 800
    desired_height = 600
    height, width, channels = image.shape
    ratio_width = desired_width / width
    ratio_height = desired_height / height
    ratio = min(ratio_width, ratio_height)
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    resized_img = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    cv2.imshow('Chessboard', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No corners were found")

701 3038
801 1677
1416 2278
1416 2594
1653 2598
1663 2122
1778 2485
1780 2320
1777 2650
1790 1984
1791 1807
2616 1731
2482 3481


In [25]:
object_points_plane1 = required_world_coordinates[4:]
image_points_plane1 = required_image_coordinates[4:]
object_points_plane1_ = [object_points_plane1.reshape(-1, 1, 3).astype(np.float32)]
image_points_plane1_ = [image_points_plane1.reshape(-1, 1, 2).astype(np.float32)]
# print(object_points_plane1_)
# print(image_points_plane1_)
dist_coeffs = np.zeros((5, 1), dtype=np.float32)
camera_matrix = np.array([
    [gray_image.shape[::-1][0], 0, gray_image.shape[::-1][0]/2],
    [0, gray_image.shape[::-1][1], gray_image.shape[::-1][1]/2],
    [0, 0, 1]
], dtype=np.float32)
ret, camera_matrix, _, rvecs, tvecs = cv2.calibrateCamera(
    object_points_plane1_,
    image_points_plane1_,
    gray_image.shape[::-1],
    camera_matrix,
    dist_coeffs,
    flags=cv2.CALIB_USE_INTRINSIC_GUESS | cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | 
          cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_K6
)
object_points_plane2 = required_world_coordinates[:4]
image_points_plane2 = required_image_coordinates[:4]
all_object_points = np.concatenate((object_points_plane1, object_points_plane2), axis=0)
all_image_points = np.concatenate((image_points_plane1, image_points_plane2), axis=0)
all_object_points_ = [all_object_points.reshape(-1, 1, 3).astype(np.float32)]
all_image_points_ = [all_image_points.reshape(-1, 1, 2).astype(np.float32)]
# print(all_object_points_)
# print(all_image_points_)
ret, camera_matrix, _, rvecs, tvecs = cv2.calibrateCamera(
    all_object_points_,
    all_image_points_,
    gray_image.shape[::-1],
    camera_matrix,
    dist_coeffs,
    flags=cv2.CALIB_USE_INTRINSIC_GUESS | cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | 
          cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_K6
)
projection_matrix = []
for rvec, tvec in zip(rvecs, tvecs):
        R, _ = cv2.Rodrigues(rvec)
        extrinsic_matrix = np.hstack((R, tvec))
        projection_matrix = np.dot(camera_matrix, extrinsic_matrix)
print(camera_matrix, dist_coeffs, projection_matrix)
hh = projection_matrix @ np.array([0, 4, 2, 1])
print(hh[0]/hh[2], hh[1]/hh[2])

[[3.63677643e+03 0.00000000e+00 1.47706423e+03]
 [0.00000000e+00 3.62025059e+03 2.13304171e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[0.]
 [0.]
 [0.]
 [0.]
 [0.]] [[ 1.43251747e+03 -4.01715560e+02 -3.63240689e+03  6.98520146e+04]
 [-8.16727077e+02 -4.05461157e+03 -7.41055432e+02  1.15818149e+05]
 [-7.06938114e-01 -2.63733743e-01 -6.56264441e-01  4.53800311e+01]]
1417.7330607705924 2281.1377715777144


In [26]:
P = projection_matrix

wireframe_points_world = np.array([
    [14, 12, 0],
    [0, 12, 0],
    [14, -6, 0],
    [0, -6, 0]
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(image, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
wireframe_points_world = np.array([
    [0, 12, 14],
    [0, 12, 0],
    [0, -6, 14],
    [0, -6, 0]
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(image, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
desired_width = 800
desired_height = 600
height, width, channels = image.shape
ratio_width = desired_width / width
ratio_height = desired_height / height
ratio = min(ratio_width, ratio_height)
new_width = int(width * ratio)
new_height = int(height * ratio)
resized_img = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
cv2.imshow('Chessboard', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Explaination:
We had to calibrate twice in the case of the inbuilt function beacause it basically calibrates for objects in the sam plane. The calibration for both the inbuilt function and the self inplimented function are a bit distorted.

# Q2.2

In [28]:
img = cv2.imread('assign1.jpg')
number_of_squares_X = 5
number_of_squares_Y = 5
square_size = 2.0

object_points = np.zeros((number_of_squares_X * number_of_squares_Y, 3), np.float32)
object_points[:, :2] = np.mgrid[0:number_of_squares_X, 0:number_of_squares_Y].T.reshape(-1, 2)
object_points = object_points * square_size

object_points_array = []
image_points_array = []

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (number_of_squares_X, number_of_squares_Y), None)
if ret:
    object_points_array.append(object_points)
    corners_corrected = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    image_points_array.append(corners_corrected)

    img = cv2.drawChessboardCorners(img, (number_of_squares_X, number_of_squares_Y), corners_corrected, ret)

    desired_width = 800
    desired_height = 600
    height, width, channels = img.shape
    ratio_width = desired_width / width
    ratio_height = desired_height / height
    ratio = min(ratio_width, ratio_height)
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    cv2.imshow('Chessboard', resized_img)
    cv2.waitKey(0)
else:
    print("fuck")
cv2.destroyAllWindows()

if len(object_points_array) > 0:
    camera_matrix = np.array([[gray.shape[1], 0, gray.shape[1]/2],
                          [0, gray.shape[0], gray.shape[0]/2],
                          [0, 0, 1]], dtype=float)
    dist_coeffs = np.zeros((5, 1))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points_array, image_points_array, gray.shape[::-1], camera_matrix, dist_coeffs, flags=cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_ZERO_TANGENT_DIST + cv2.CALIB_FIX_K1 + cv2.CALIB_FIX_K2 + cv2.CALIB_FIX_K3 + cv2.CALIB_FIX_K4 + cv2.CALIB_FIX_K5 + cv2.CALIB_FIX_K6)
    projection_matrices = []
    for rvec, tvec in zip(rvecs, tvecs):
        R, _ = cv2.Rodrigues(rvec)
        extrinsic_matrix = np.hstack((R, tvec))
        projection_matrix = np.dot(mtx, extrinsic_matrix)
        projection_matrices.append(projection_matrix)
    print("Camera matrix:\n", mtx)
    print("Distortion coefficients:\n", dist)
    print("Rotation Vectors:\n", rvecs)
    print("Translation Vectors:\n", tvecs)
    print("Projection Matrix:\n", projection_matrices)

Camera matrix:
 [[3.18376820e+03 0.00000000e+00 1.06286689e+03]
 [0.00000000e+00 2.28507983e+03 1.02256268e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Rotation Vectors:
 (array([[-1.00917651],
       [-0.01888391],
       [ 0.00621452]]),)
Translation Vectors:
 (array([[-3.97906086],
       [-5.6893279 ],
       [65.061719  ]]),)
Projection Matrix:
 [array([[ 3.19696368e+03, -8.88343113e+02,  5.06296037e+02,
         5.64835392e+04],
       [ 4.51465780e+01,  3.51448331e+02,  2.47823932e+03,
         5.35291173e+04],
       [ 1.29583475e-02, -8.46387493e-01,  5.32409892e-01,
         6.50617190e+01]])]


In [30]:
R, _ = cv2.Rodrigues(rvecs[0])
t = tvecs[0]
P = np.matmul(mtx, np.hstack((R, t)))
wireframe_points_world = np.array([
    [0, 0, 0],
    [0, (number_of_squares_Y - 1) * square_size, 0],
    [(number_of_squares_X - 1) * square_size, 0, 0],
    [(number_of_squares_X - 1) * square_size, (number_of_squares_Y - 1) * square_size, 0],
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
patch_coordinates = wireframe_points_image
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(img, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
desired_width = 800
desired_height = 600
height, width, channels = img.shape
ratio_width = desired_width / width
ratio_height = desired_height / height
ratio = min(ratio_width, ratio_height)
new_width = int(width * ratio)
new_height = int(height * ratio)
resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
cv2.imshow('Wireframe Overlay', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Explaination:
The calibration seems to be perfect in this case bacause I used the all the chessboard coordinates for calibration. But ideally, the calibration in case of objects in one plane can be a bit distorted.

# Q2.3

In [31]:
print((P @ [0, 0, 0, 1])[0]/(P @ [0, 0, 0, 1])[2], (P @ [0, 0, 0, 1])[1]/(P @ [0, 0, 0, 1])[2])

868.1531944135427 822.743667274874


### Explaination:
Yes this result aggrees with my observation. Since, the world coordinate i choose was corresponding to these pixel values only.

# Q3.1

In [32]:
R, _ = cv2.Rodrigues(rvecs[0])
t = tvecs[0]
P = np.matmul(mtx, np.hstack((R, t)))
wireframe_points_world = np.array([
    [0+10, 0, 0],
    [0+10, (number_of_squares_Y - 1) * square_size, 0],
    [(number_of_squares_X - 1) * square_size+10, 0, 0],
    [(number_of_squares_X - 1) * square_size+10, (number_of_squares_Y - 1) * square_size, 0],
], dtype=float)
wireframe_points_world_homogeneous = np.hstack((wireframe_points_world, np.ones((4, 1))))
wireframe_points_image_homogeneous = np.dot(P, wireframe_points_world_homogeneous.T).T
wireframe_points_image = (wireframe_points_image_homogeneous / wireframe_points_image_homogeneous[:, -1].reshape(-1, 1))[:, :2]
for start, end in zip([0, 0, 1, 3], [1, 2, 3, 2]):
    cv2.line(img, tuple(wireframe_points_image[start].astype(int)), tuple(wireframe_points_image[end].astype(int)), (0, 255, 0), 3)
desired_width = 800
desired_height = 600
height, width, channels = img.shape
ratio_width = desired_width / width
ratio_height = desired_height / height
ratio = min(ratio_width, ratio_height)
new_width = int(width * ratio)
new_height = int(height * ratio)
resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
cv2.imshow('Wireframe Overlay', resized_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Explaination:
Yes the frame looks consistant. Its a bit skewed as expected.

# Q3.2

In [33]:
def is_point_in_poly(x, y, poly):
    num = len(poly)
    j = num - 1
    c = False
    for i in range(num):
        if ((poly[i][1] > y) != (poly[j][1] > y)) and (x < poly[i][0] + (poly[j][0] - poly[i][0]) * (y - poly[i][1]) / (poly[j][1] - poly[i][1])):
            c = not c
        j = i
    return c

def iterate_polygon_points(polygon):
    minX = min(polygon, key=lambda p: p[0])[0]
    maxX = max(polygon, key=lambda p: p[0])[0]
    minY = min(polygon, key=lambda p: p[1])[1]
    maxY = max(polygon, key=lambda p: p[1])[1]
    output = []
    for x in range(minX, maxX + 1):
        for y in range(minY, maxY + 1):
            if is_point_in_poly(x, y, polygon):
                output.append([x, y])
    return output
polygon = patch_coordinates.astype(int)
temp = polygon[1].copy()
polygon[1] = polygon[2].copy()
polygon[2] = polygon[3].copy()
polygon[3] = temp.copy()
patch_pixel_coordinates = iterate_polygon_points(polygon)

In [34]:
imagee = cv2.imread("assign1.jpg")
image = imagee.copy()
R_copy = R.copy()
R_copy[:, 2] = np.array([t[0][0], t[1][0], t[2][0]])
K_inv = np.linalg.inv(mtx)
R_inv = np.linalg.inv(R_copy)
P_inv = R_inv @ K_inv
for u, v in patch_pixel_coordinates:
        world_coordinates = P_inv @ np.array([u, v, 1])
        world_coordinates = [world_coordinates[0]/world_coordinates[2], world_coordinates[1]/world_coordinates[2], 0]
        new_world_coordinates = world_coordinates
        new_world_coordinates[0] += 10
        new_world_coordinates.append(1)
        point_image_homogeneous = np.dot(P, new_world_coordinates)
        point_image_new = point_image_homogeneous[:2] / point_image_homogeneous[2]
        image[int(point_image_new[1]), int(point_image_new[0])] = image[int(v), int(u)].copy()
desired_width = 800
desired_height = 600
height, width, channels = image.shape
ratio_width = desired_width / width
ratio_height = desired_height / height
ratio = min(ratio_width, ratio_height)
new_width = int(width * ratio)
new_height = int(height * ratio)
resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
cv2.imshow('Wireframe Overlay', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Explaination:
Yes the overlay seems to be perfect and fit the original wireframe completely.